In [100]:
%%latex
$$
H=\Large\frac{C}{n1*m1+n2*m2+n3*m3+n4*m4}\\
\begin{aligned}
& \text {Valores de $n_{i}$ en los distintos casos del modelo de ajuste }\\
&\begin{array}{cccc}
\hline \hline \text { producto 1 } & \text { Producto2 } & \text { Producto 3 } & \text { Producto 4 } \\
\hline 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
1 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
1 & 0 & 1 & 0 \\
0 & 1 & 1 & 0 \\
1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 \\
1 & 0 & 0 & 1 \\
0 & 1 & 0 & 1 \\
1 & 1 & 0 & 1 \\
0 & 0 & 1 & 1 \\
1 & 0 & 1 & 1 \\
0 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 \\
\hline
\end{array}
\end{aligned}
$$

<IPython.core.display.Latex object>

In [101]:
import numpy as np
import math
def model_calc(m1,m2,m3,m4,n1,n2,n3,n4,C,c_min,h):
    if n1==0 and n2==0 and n3==0 and n4==0:
        print("no todos los tanques pueden estar vacios, no se puede resolver")
    #n1=1
    #n2=1
    #n3=1
    #n4=1
    h=C/(n1*m1+n2*m2+n3*m3+n4*m4)
    product_1=n1*m1*h
    product_2=n2*m2*h
    product_3=n3*m3*h
    product_4=n4*m4*h
    products=np.array([product_1,product_2,product_3,product_4])
    tank_fill=products/c_min
    return h, tank_fill

In [102]:
def round_loading(tank_fill_,posibilities):
    #print(posibilities)
    if posibilities[0]==1.0:
        tank_fill_[0]=math.ceil(tank_fill_[0])
    if posibilities[0]==0.0:
        tank_fill_[0]=math.floor(tank_fill_[0])
    if posibilities[1]==1.0:
        tank_fill_[1]=math.ceil(tank_fill_[1])
    if posibilities[1]==0.0:
        tank_fill_[1]=math.floor(tank_fill_[1])
    if posibilities[2]==1.0:
        tank_fill_[2]=math.ceil(tank_fill_[2])
    if posibilities[2]==0.0:
        tank_fill_[2]=math.floor(tank_fill_[2])
    if posibilities[3]==1.0:
        tank_fill_[3]=math.ceil(tank_fill_[3])
    if posibilities[3]==0.0:
        tank_fill_[3]=math.floor(tank_fill_[3])
    return tank_fill_

In [103]:
def truck_loading_posibilities(m0,m1,m2,m3,C,c_min,h):
    i=0 #contador
    columns_names= ['flow_1','flow_2','flow_3','flow_4','min_after','product_1','product_2','product_3','product_4']
    results=pd.DataFrame(columns = columns_names)
    posibilities=np.array([[0,0,0,0],[1,0,0,0],[0,1,0,0],[1,1,0,0],[0,0,1,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]])
    for iteration in range(1,len(posibilities)):
        for pos in range(0,len(posibilities)):
            h,tank_fill_=model_calc(m0,m1,m2,m3,posibilities[iteration,0],posibilities[iteration,1],posibilities[iteration,2],posibilities[iteration,3],C,c_min,h)
            tank_fill_fixed=round_loading(tank_fill_,posibilities[pos,:])
            if sum(tank_fill_fixed)==4.0:
                results.loc[i]=[m0, m1, m2, m3, round((h),2), int(tank_fill_fixed[0]), int(tank_fill_fixed[1]), int(tank_fill_fixed[2]), int(tank_fill_fixed[3])]
                #print(model_calc(m0,m1,m2,m3,posibilities[iteration,0],posibilities[iteration,1],posibilities[iteration,2],posibilities[iteration,3],C,c_min,h))
                i=i+1
    return results.drop_duplicates()

In [104]:
#agregar el control de la ventana horaria a esta funcion
def stock_overfill_check(c0_max, c1_max, c2_max, c3_max, q0,q1,q2,q3,m0,m1,m2,m3,c_min,results):
    i=0
    columns_names= ['flow_1','flow_2','flow_3','flow_4','min_after','product_1','product_2','product_3','product_4']
    results_filtered=pd.DataFrame(columns = columns_names)
    for index, row in results.iterrows():
        cap0=c0_max-q0+m0*row['min_after']
        cap1=c1_max-q1+m1*row['min_after']
        cap2=c2_max-q2+m2*row['min_after']
        cap3=c3_max-q3+m3*row['min_after']
        diff_0=cap0-row['product_1']*c_min   #cheque si los valores entre lo que puedo llenar y lo que lleno son positivos, si son negativos no va la solucion
        diff_1=cap1-row['product_2']*c_min
        diff_2=cap2-row['product_3']*c_min
        diff_3=cap3-row['product_4']*c_min
        if (diff_0 < 0 or diff_1 < 0 or diff_2 < 0 or diff_3 < 0):
            pass
            #print("solucion no valida, se supera la capacidad maxima del tanque")
            #print(diff_0)
            #print(diff_1)
            #print(diff_2)
            #print(diff_3)
        else:
            results_filtered.loc[i]=[m0, m1, m2, m3, row['min_after'], row['product_1'], row['product_2'], row['product_3'], row['product_4']]
            i=i+1
    return results_filtered

In [105]:
#agregar el control de la ventana horaria 
def stock_underfill_check(c0_max, c1_max, c2_max, c3_max, q0,q1,q2,q3,m0,m1,m2,m3,c_min,results):
    i=0
    columns_names= ['flow_1','flow_2','flow_3','flow_4','min_after','product_1','product_2','product_3','product_4']
    results_filtered=pd.DataFrame(columns = columns_names)
    for index, row in results.iterrows():
        cap0=q0-m0*row['min_after']
        cap1=q1-m1*row['min_after']
        cap2=q2-m2*row['min_after']
        cap3=q3-m3*row['min_after']
        if (cap0 < 0 or cap1 < 0 or cap2 < 0 or cap3 < 0):
            pass
        else:
            results_filtered.loc[i]=[m0, m1, m2, m3, row['min_after'], row['product_1'], row['product_2'], row['product_3'], row['product_4']]
            i=i+1
    return results_filtered


In [106]:
def check_window_time(results,windows, m0, m1, m2, m3,c_min,h):
    i=0
    columns_names= ['H_calculated','Real_H','Product_1','Product_2','Product_3', 'Product_4']
    H=pd.DataFrame(columns = columns_names)
    for index, row in results.iterrows():
        N1=row['product_1']*c_min
        N2=row['product_2']*c_min
        N3=row['product_3']*c_min
        N4=row['product_4']*c_min
        C_cal=N1+N2+N3+N4 
        h_, results=model_calc(m0,m1,m2,m3,1,1,1,1,C_cal,c_min,h)
        H.loc[i]=[row['min_after'],h_, row['product_1'], row['product_2'], row['product_3'], row['product_4']]
        i=i+1
    return H


In [107]:
#Ver pedidos en curso
#cantidad de litros por minuto sale ddel modelo de ajuste o del scm
def eval_model(m0, m1, m2, m3, C, c_min, c0_max, c1_max, c2_max, c3_max, window, q0, q1, q2, q3):

    #evaluar modelo
    results_all=pd.DataFrame()
    for w in range(1,200):
        results=truck_loading_posibilities(m0,m1,m2,m3, (0.5+w/200)*C,c_min,h_max) #calculo de estimacion de ventas y modelo de carga
        results1=stock_overfill_check(c0_max, c1_max, c2_max, c3_max, q0,q1,q2,q3,m0,m1,m2,m3,c_min,results) #no pasar del maximo
        results2=stock_underfill_check(c0_max, c1_max, c2_max, c3_max, q0,q1,q2,q3,m0,m1,m2,m3,c_min,results1) #no quebrarstock
        H=check_window_time(results2,0,m0,m1,m2,m3,c_min,h_max)
        H['diff']=H['H_calculated']-H['Real_H']
        H['diff_window']= window-H['Real_H']
        H['product_1_liters']=H['Product_1']*c_min
        H['product_2_liters']=H['Product_2']*c_min
        H['product_3_liters']=H['Product_3']*c_min
        H['product_4_liters']=H['Product_4']*c_min
        results_all=results_all.append(H, ignore_index=True)
    return results_all
#results1.head(1000)

In [112]:
m0=3
m1=1
m2=1
m3=0.5
#capacidad total cel camion en litros
C=20000
#capacidad minima de los tanques sdel camion (considerando que son todos iguales)
c_min=5000
#cantidad de minutos analizados en el modelo
h_max=10000
#capacidad maxima de los tanques de la estacion
c0_max=20000
c1_max=15000
c2_max=15000
c3_max=10000
window=5000
#stock de la estacion en cada no de los tanques. Si hay dos tanques que tenga el mismo prodcuto va la suma
q0=20000
q1=15000
q2=15000
q3=10000
columns_names= ['Hora_entrega','q0','q1','q2','q3','p0','p1','p2','p3']
sim_data=pd.DataFrame(columns = columns_names)
for i in range(1,10):
    results_all=eval_model(m0, m1, m2, m3, C, c_min, c0_max, c1_max, c2_max, c3_max, window, q0, q1, q2, q3)
    results_all['abs_diff']=abs(results_all['diff'])
    results_optimized_time=results_all[results_all['abs_diff']==results_all['abs_diff'].min()].iloc[:1]  #elijo la primer solucion puede haber varias iguales
    print("Ventana_temporal: "+ str((results_optimized_time['H_calculated'].iloc[0])/60))
    print("valor de stock inicial de q0: " + str(q0))
    q0=q0-m0*results_optimized_time['H_calculated'].iloc[0]
    print("valor de stock despues de las ventas de q0: "+ str(q0))
    q0=q0+results_optimized_time['product_1_liters'].iloc[0]
    print("valor despues de reponer el stock con la condicion de minimo tiempo: " + str(q0))
    print("----------------------------------------------------")
    print("valor de stock inicial de q1: " + str(q1))
    q1=q1-m1*results_optimized_time['H_calculated'].iloc[0]
    print("valor de stock despues de las ventas de q1: "+ str(q1))
    q1=q1+results_optimized_time['product_2_liters'].iloc[0]
    print("valor despues de reponer el stock con la condicion de minimo tiempo: " + str(q1))
    print("----------------------------------------------------")
    print("valor de stock inicial de q2: " + str(q2))
    q2=q2-m2*results_optimized_time['H_calculated'].iloc[0]
    print("valor de stock despues de las ventas de q2: "+ str(q2))
    q2=q2+results_optimized_time['product_3_liters'].iloc[0]
    print("valor despues de reponer el stock con la condicion de minimo tiempo: " + str(q2))
    print("----------------------------------------------------")
    print("valor de stock inicial de q3: " + str(q3))
    q3=q3-m3*results_optimized_time['H_calculated'].iloc[0]
    print("valor de stock despues de las ventas de q3: "+ str(q3))
    q3=q3+results_optimized_time['product_4_liters'].iloc[0]
    print("valor despues de reponer el stock con la condicion de minimo tiempo: " + str(q3))
    sim_data.loc[i]=[(results_optimized_time['H_calculated'].iloc[0])/60,q0,q1,q2,q3,results_optimized_time['product_1_liters'],results_optimized_time['product_2_liters'],results_optimized_time['product_3_liters'],results_optimized_time['product_4_liters']]
    i=i+1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ventana_temporal: 60.606
valor de stock inicial de q0: 19545.519999999997
valor de stock despues de las ventas de q0: 8636.439999999997
valor despues de reponer el stock con la condicion de minimo tiempo: 18636.439999999995
----------------------------------------------------
valor de stock inicial de q1: 13181.839999999997
valor de stock despues de las ventas de q1: 9545.479999999996
valor despues de reponer el stock con la condicion de minimo tiempo: 14545.479999999996
----------------------------------------------------
valor de stock inicial de q2: 8181.8399999999965
valor de stock despues de las ventas de q2: 4545.479999999996
valor despues de reponer el stock con la condicion de minimo tiempo: 9545.479999999996
----------------------------------------------------
valor de stock inicial de q3: 4090.9199999999983
valor de stock despues de las ventas de q3: 2272.739999999998
valor despues de reponer el stock con la condicion de minimo tiempo: 2272.739999999998


,H_calculated,Real_H,Product_1,Product_2,Product_3,Product_4,diff,diff_window,product_1_liters,product_2_liters,product_3_liters,product_4_liters,abs_diff
0,25000.0,25000.0,1.0,2.0,1.0,0.0,-3.637979e-12,-20000.0,5000.0,10000.0,5000.0,0.0,3.637979e-12
1,25000.0,25000.0,1.0,1.0,2.0,0.0,-3.637979e-12,-20000.0,5000.0,5000.0,10000.0,0.0,3.637979e-12
2,25000.0,25000.0,1.0,1.0,1.0,1.0,-3.637979e-12,-20000.0,5000.0,5000.0,5000.0,5000.0,3.637979e-12
